# Day 16 - Maze - part 2

Need to find all the best paths, and mark them.

I will modify in such a way that the shortest paths are stored per node.

Will need a new dictionary with a list of paths.

# Read file as lines

In [ ]:
TEST = True

# INPUT_FN = "input_test.txt"
# TEST_SOLUTION = 45

INPUT_FN = "input_test2.txt"
TEST_SOLUTION = 64

if not TEST:
    INPUT_FN = "input.txt"

with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

In [2]:
lines

['###############',
 '#.......#....E#',
 '#.#.###.#.###.#',
 '#.....#.#...#.#',
 '#.###.#####.#.#',
 '#.#.#.......#.#',
 '#.#.#####.###.#',
 '#...........#.#',
 '###.#.#####.#.#',
 '#...#.....#.#.#',
 '#.#.#.###.#.#.#',
 '#.....#...#.#.#',
 '#.###.#.#.#.#.#',
 '#S..#.....#...#',
 '###############']

In [3]:
MAP=lines

Find start and end

In [4]:
start_pos=None
end_pos = None

for r0,l0 in enumerate(lines):
    if 'S' in l0:
        start_pos = (r0,l0.index('S'))
    if 'E' in l0:
        end_pos = (r0,l0.index('E'))


In [5]:
print(f"{start_pos=}, {end_pos=}")

start_pos=(13, 1), end_pos=(1, 13)


In [6]:
def printd(s):
    if TEST:
        print(s)

Suggested algorithm to solve

BFS whereby visited locations must include the direction of travel

Store the lowest score for each location visited. Have a dictionary to store these.

In [12]:
from collections import deque


best_scores = dict()
visited = []
robot_pos = (*start_pos, 0,+1)
best_scores[robot_pos]= 0

bfs_q = deque()
bfs_q.append(robot_pos)

best_paths_nodes=dict()
best_paths_nodes[robot_pos]=set()

iter=0

while True:
    printd(f"{iter=} , {bfs_q=}")
    if len(bfs_q)==0:
        # no more nodes to visit
        printd("No more nodes to visit")
        break

    curr_pos = bfs_q.popleft()
    curr_score = best_scores[curr_pos]
    cur_best_paths_nodes = best_paths_nodes[curr_pos] # should be a set

    printd(f"{curr_pos=} , {curr_score=}, {cur_best_paths_nodes=}")

    #Check is end position
    if MAP[curr_pos[0]][curr_pos[1]] == 'E':
        printd("End node")
        continue # skip the rest

    #Options are, to move forward or to turn clockwise or anticlockwise
    new_poss = []

    #move forward need to check if there is a wall at new position
    try_pos = (curr_pos[0]+curr_pos[2], curr_pos[1]+curr_pos[3])
    printd(f"move forward to {try_pos=} ?")

    if MAP[try_pos[0]][try_pos[1]] != '#':
        # ok
        printd("can move forward")
        new_poss.append( ( (*try_pos, curr_pos[2], curr_pos[3]) , curr_score+1) )
    
    # rotations
    new_poss.append( ((curr_pos[0], curr_pos[1], -curr_pos[3], curr_pos[2]), curr_score+1000) ) # clockwise
    new_poss.append( ((curr_pos[0], curr_pos[1], curr_pos[3], -curr_pos[2]) , curr_score+1000) ) # anti-clockwise

    # for all these new positions to consider, check if they have been visited
    for npos0,sc0 in new_poss:
        #position visited?

        path_nodes_try = cur_best_paths_nodes.copy()
        path_nodes_try.add(npos0)

        if npos0 in visited:
            # compare scores
            bsc0 = best_scores[npos0]
            b_pn0 = best_paths_nodes[npos0]
            
            if sc0<bsc0:
                printd(f"This path has a better score that previously calculated {sc0=} < {bsc0=}")
                best_scores[npos0] = sc0
                # re-add position to queue to recalculate any scores after this
                bfs_q.append(npos0)

                # best path nodes update
                # This score is better, so replace
                best_paths_nodes[npos0] = path_nodes_try
                
            elif sc0==bsc0:
                # don't need to change scored but does union of the nodes
                best_paths_nodes[npos0] = b_pn0.union(path_nodes_try)

                printd(f"Same score {sc0=} < {bsc0=}, joining path nodes from each")

        else:
            #add this as a new visited place
            printd(f"New visit with {npos0=}")
            visited.append(npos0)
            best_scores[npos0] = sc0
            bfs_q.append(npos0)

            best_paths_nodes[npos0] = path_nodes_try

    iter+=1

iter=0 , bfs_q=deque([(13, 1, 0, 1)])
curr_pos=(13, 1, 0, 1) , curr_score=0, cur_best_paths_nodes=set()
move forward to try_pos=(13, 2) ?
can move forward
New visit with npos0=(13, 2, 0, 1)
New visit with npos0=(13, 1, -1, 0)
New visit with npos0=(13, 1, 1, 0)
iter=1 , bfs_q=deque([(13, 2, 0, 1), (13, 1, -1, 0), (13, 1, 1, 0)])
curr_pos=(13, 2, 0, 1) , curr_score=1, cur_best_paths_nodes={(13, 2, 0, 1)}
move forward to try_pos=(13, 3) ?
can move forward
New visit with npos0=(13, 3, 0, 1)
New visit with npos0=(13, 2, -1, 0)
New visit with npos0=(13, 2, 1, 0)
iter=2 , bfs_q=deque([(13, 1, -1, 0), (13, 1, 1, 0), (13, 3, 0, 1), (13, 2, -1, 0), (13, 2, 1, 0)])
curr_pos=(13, 1, -1, 0) , curr_score=1000, cur_best_paths_nodes={(13, 1, -1, 0)}
move forward to try_pos=(12, 1) ?
can move forward
New visit with npos0=(12, 1, -1, 0)
New visit with npos0=(13, 1, 0, -1)
New visit with npos0=(13, 1, 0, 1)
iter=3 , bfs_q=deque([(13, 1, 1, 0), (13, 3, 0, 1), (13, 2, -1, 0), (13, 2, 1, 0), (12, 1, -1, 0),

This algorithm above does not work well. It is slow because it checks for too many paths. Dijkstra algorithm may be better suited.

There are probably many end positions in the score dictionary, with different directions. I will pick the lowest

In [19]:
best_score_and_path = None
for k,nodes0 in best_paths_nodes.items():
    if k[0] == end_pos[0] and k[1]==end_pos[1]:
        sc0 = best_scores[k]
        print(f"score {sc0}, best_paths_nodes {k=} , {nodes0=}")
        
        if best_score_and_paths is None:
            best_score_and_paths = (sc0, nodes0)
        else:
            if sc0< best_score_and_paths[0]:
                best_score_and_paths = (sc0,nodes0)
        
print(f"{best_score_and_paths}")

score 10028, best_paths_nodes k=(1, 13, 0, 1) , nodes0={(10, 1, -1, 0), (5, 11, 0, 1), (7, 4, 0, 1), (10, 3, -1, 0), (7, 3, -1, 0), (1, 9, 0, 1), (5, 9, -1, 0), (13, 1, -1, 0), (2, 9, -1, 0), (3, 11, -1, 0), (11, 5, 0, 1), (7, 6, 0, 1), (1, 11, 0, 1), (5, 11, -1, 0), (9, 1, 0, 1), (3, 10, 0, -1), (11, 5, -1, 0), (8, 3, -1, 0), (9, 2, 0, 1), (7, 8, 0, 1), (1, 9, -1, 0), (9, 3, 0, 1), (9, 5, -1, 0), (12, 1, -1, 0), (11, 1, 0, 1), (7, 9, 0, 1), (4, 11, -1, 0), (8, 5, -1, 0), (11, 3, 0, 1), (9, 1, -1, 0), (7, 9, -1, 0), (5, 10, 0, 1), (11, 4, 0, 1), (7, 5, 0, 1), (11, 1, -1, 0), (3, 9, 0, -1), (9, 3, -1, 0), (10, 5, -1, 0), (7, 5, -1, 0), (7, 7, 0, 1), (11, 3, -1, 0), (6, 9, -1, 0), (5, 9, 0, 1), (3, 11, 0, -1), (3, 9, -1, 0), (1, 10, 0, 1), (1, 12, 0, 1), (1, 13, 0, 1), (11, 2, 0, 1), (7, 3, 0, 1)}
score 7036, best_paths_nodes k=(1, 13, -1, 0) , nodes0={(10, 1, -1, 0), (7, 4, 0, 1), (7, 10, 0, 1), (10, 3, -1, 0), (7, 3, -1, 0), (13, 1, -1, 0), (7, 13, -1, 0), (4, 13, -1, 0), (1, 13, -1, 0

In [20]:
best_nodes_coords = { n0[:2] for n0 in best_score_and_paths[1]}

In [21]:
score = len(best_nodes_coords) 

In [22]:
print(score)

45


In [23]:
if TEST:
    assert score == TEST_SOLUTION

Debug

In [25]:
best_nodes_coords

{(1, 13),
 (2, 13),
 (3, 13),
 (4, 13),
 (5, 13),
 (6, 13),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 13),
 (8, 3),
 (8, 5),
 (8, 11),
 (8, 13),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 5),
 (9, 11),
 (9, 13),
 (10, 1),
 (10, 3),
 (10, 5),
 (10, 11),
 (10, 13),
 (11, 1),
 (11, 2),
 (11, 3),
 (11, 4),
 (11, 5),
 (11, 11),
 (11, 13),
 (12, 1),
 (12, 11),
 (12, 13),
 (13, 1),
 (13, 11),
 (13, 12),
 (13, 13)}

In [27]:
for irow, l0 in enumerate(MAP):
    printline=[]
    for icol, c in enumerate(l0):
        if (irow,icol) in best_nodes_coords:
            printline.append('O')
        else:
            printline.append(c)
    print("".join(printline))

###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
